In [ ]:
# matplotlib plots within notebook
%matplotlib inline

import platform
print("python: "+platform.python_version())


import numpy as np
import matplotlib.pyplot as plt

import time

import os, shutil, sys

import pydicom as dicom

from scipy import ndimage
from skimage import morphology
from skimage import measure

import pandas as pd

sys.path.insert(0, '.')
from DeepAttCorr_lib import GAN_3D_lib as GAN
from DeepAttCorr_lib import data_handling as DH
from DeepAttCorr_lib import file_manage_utils as File_mng

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
print('Using TensorFlow version: '+tf.__version__)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# Paths and Definitions

In [ ]:
# Output path of generated dataframe
OUTPUT_PATH = './test_outputs'

# --------------------------------------------------------------
# -------------------- Choose the model ------------------------
# --------------------------------------------------------------

# -------------- Sucessfull topologies

#-- 3D U-Net (Sup. Pre-Train No GAN.Loss.)
TEST_NAME = '3D_UNet'
CHECKPOINT_PATH = './trained_models/Sup_Pre_Train_No_GAN_Loss/'
CHECKPOINT_NAME = 'Sup_Pre_Train_No_GAN_Loss'

# #-- Sup. Pre-Train RestrictedGrad.
# TEST_NAME = 'Sup_Pre_Train_RestrictedGrad'
# CHECKPOINT_PATH = './trained_models/Sup_Pre_Train_RestrictedGrad'
# CHECKPOINT_NAME = 'Sup_Pre_Train_RestrictedGrad_generator_model'

# -------------- Other tests

# #-- Baseline
# TEST_NAME = 'Baseline'
# CHECKPOINT_PATH = './trained_models/Baseline'
# CHECKPOINT_NAME = 'Baseline_generator_model'

# #-- Sup. Pre-Train
# TEST_NAME = 'Sup_Pre_Train'
# CHECKPOINT_PATH = './trained_models/Sup_Pre_Train'
# CHECKPOINT_NAME = 'Sup_Pre_Train_generator_model'

# #-- Sup. No Pre-Train
# TEST_NAME = 'Sup_Not_Pre_Train'
# CHECKPOINT_PATH = './trained_models/Sup_Not_Pre_Train'
# CHECKPOINT_NAME = 'Sup_Not_Pre_Train_generator_model'

# #-- Progressive Growing Pix2Pix GAN
# TEST_NAME = 'ProGAN_Pix2Pix_3D'
# CHECKPOINT_PATH = './trained_models/ProGAN_Pix2Pix_3D'
# CHECKPOINT_NAME = 'ProGAN_Pix2Pix_3D_generator_model'


# --------------------------------------------------------------
# -------------------- Choose the dataset ----------------------
# --------------------------------------------------------------

# DATASET_NAME = 'HNSCC_Validation_Samples'
DATASET_NAME = 'NSCLC Radiogenomics'
# DATASET_NAME = 'TCGA-HNSC'
# DATASET_NAME = 'TCGA-LUAD'
# DATASET_NAME = 'TCGA-THCA'
# DATASET_NAME = 'CPTAC-LUAD'
# DATASET_NAME = 'CPTAC-PDA'
# DATASET_NAME = 'CPTAC-UCEC'
# DATASET_NAME = 'CPTAC-LSCC'



### Input data info

In [ ]:
# This data is from the "Generate_train_and_validation_dataset" notebook:
# Objective size of the volume
Objective_size = [256, 256, 512]
# Voxel size used in the dataset (and hence the NN)
Voxel_size_NORM = [2.734375, 2.734375, 2.5473046004770703]

# Set different thresholds for the segmentation of the CT images
# Thresholds are in Hounsfield Units
HU_Air_limit = -1000
HU_Lung_upper_limit = -500
HU_Fluids_Fat_lower_limit = -125
HU_Fluids_Fat_upper_limit = 10.0
HU_Parenchyma_upper_limit = 90.0
HU_Parenchyma_lower_limit = 10.0
HU_Bone_low_limit = 90.0
HU_Bone_upper_limit = 1300


# Set-up

In [ ]:
# Build paths
DATASET_PATH = os.path.join('./datasets/DICOM/',DATASET_NAME)
OUTPUT_PATH = os.path.join(OUTPUT_PATH, TEST_NAME)
OUTPUT_PATH = os.path.join(OUTPUT_PATH, DATASET_NAME)

File_mng.check_create_path('OUTPUT_PATH', OUTPUT_PATH, clear_folder=False)

### Load Model

In [ ]:
custom_layers_dict = {'HeScale': GAN.layers.HeScale,
                      'BiasLayer': GAN.layers.BiasLayer,
                      'PixelNormalization': GAN.layers.PixelNormalization}
synth_model = GAN.train_support.load_model(CHECKPOINT_PATH, 
                                           CHECKPOINT_NAME, 
                                           custom_obj_dict = custom_layers_dict)

In [ ]:
# Set input volume size
input_shape_list = synth_model.input.shape.as_list()

voxels_X = input_shape_list[1]
voxels_Y = input_shape_list[2]
voxels_Z = input_shape_list[3]
input_size = (voxels_X,voxels_Y,voxels_Z)

# Check if it is a segmentator model also
output_shape_list = synth_model.output.shape.as_list()
SEGM_NET = False
if output_shape_list[-1] > 1:
    print('Segmentation output found.')
    SEGM_NET = True
    
    
# Data full size
voxels_X_Full = int(input_shape_list[1])
voxels_Y_Full = int(input_shape_list[2])
voxels_Z_Full = int(Objective_size[2]*(voxels_X_Full/Objective_size[0]))
input_size_Full = (voxels_X_Full,voxels_Y_Full,voxels_Z_Full)

In [ ]:
synth_model.summary()

In [ ]:
tf.keras.utils.plot_model(synth_model, 
                          to_file=os.path.join(OUTPUT_PATH,TEST_NAME+'.png'), 
                          show_shapes=True, 
                          show_layer_names=False,
                          rankdir='TB', 
                          expand_nested=True)

# Load and Preprocess Samples

Here all the DICOM files in the dataset will be loaded and preprocessed to be tested with the model.

### Build Dicom Paths

In [ ]:
PathDicom_list = list()
DicomNameList = list()

list_paths = sorted(os.listdir(DATASET_PATH))

# Explore each series
for idx, series_name in enumerate(list_paths):
    
    print(series_name)
    DicomNameList.append(series_name)
    
    path_series = os.path.join(DATASET_PATH,series_name)
    
    # Explore each study
    for study_name in os.listdir(path_series):
        print('\t%s'%study_name)
        path_study = os.path.join(path_series,study_name)
        PathDicom_list.append(path_study)

In [ ]:
# Reserve memory
num_test_samples = len(PathDicom_list)

NAC_PET_images = np.zeros((num_test_samples,voxels_X_Full,voxels_Y_Full,voxels_Z_Full))
CT_OBJ_images = np.zeros((num_test_samples,voxels_X_Full,voxels_Y_Full,voxels_Z_Full))
test_scale_limits_list = list()

### Load DICOM

In [ ]:
# Limits margin
HU_Lung_upper_limit = HU_Lung_upper_limit
HU_Bone_upper_limit = HU_Bone_upper_limit
HU_fat_lower_limit = HU_Fluids_Fat_lower_limit * 1.2

img_count = 0
for PathDicom_test in PathDicom_list:

    print('(%d/%d) Checking path: %s'%(img_count+1,num_test_samples,PathDicom_test))

    COPY_NAC_OK = False
    COPY_CT_OK = False


    for dirName, subdirList, fileList in os.walk(PathDicom_test):

        # Scan for all image modalities and get limits
        image_path_list = list()
        nac_pet_limits = [0,0]
        ct_limits = [0,0]
        for subDirName in subdirList:
            # Build path
            PathDicom = os.path.join(dirName, subDirName)
            list_paths = sorted(os.listdir(PathDicom))

            test_slices = len(list_paths)

            # Read all slice locations
            test_locations = np.zeros((test_slices))
            for idx, filename in enumerate(list_paths):
                if ".dcm" in filename.lower():  # check whether the file's DICOM
                    RefDs = dicom.read_file(os.path.join(PathDicom,filename))
                    test_locations[idx] = np.array(RefDs.ImagePositionPatient)[2]

            # Save extrema
            if RefDs.Modality == 'PT':
                if (not 'ATTN' in RefDs.CorrectedImage) or 'NAC' in PathDicom or 'uncorrected' in PathDicom or 'NOAC' in PathDicom:
                    image_path_list.append(PathDicom)
                    nac_pet_limits[0] = test_locations.min()
                    nac_pet_limits[1] = test_locations.max()

                else:
                    image_path_list.append(PathDicom)
                    continue

            elif RefDs.Modality == 'CT':
                image_path_list.append(PathDicom)
                ct_limits[0] = test_locations.min()
                ct_limits[1] = test_locations.max()

        # Get absolute limits
        loc_min = np.max([ct_limits[0],nac_pet_limits[0]])
        loc_max = np.min([ct_limits[1],nac_pet_limits[1]])

        for PathDicom in image_path_list:

            list_paths = sorted(os.listdir(PathDicom))
            RefDs = dicom.read_file(os.path.join(PathDicom,list_paths[0]))

            IS_CT = False
            IS_PT = False
            IS_NAC_PT = False

            # Check image type
            if RefDs.Modality == 'PT':
                if (not 'ATTN' in RefDs.CorrectedImage) or 'NAC' in PathDicom or 'uncorrected' in PathDicom or 'NOAC' in PathDicom:
                    IS_NAC_PT = True
                else:
                    IS_PT = True
            elif RefDs.Modality == 'CT':
                IS_CT = True                

            if IS_PT:
                print('PET image: %d files'%(len(list_paths)))
                continue
            elif IS_NAC_PT:
                print('NAC PET image: %d files'%(len(list_paths)))
            elif IS_CT:
                print('CT image: %d files'%(len(list_paths)))



            test_cols = RefDs.Columns
            test_rows = RefDs.Rows
            test_slices = len(list_paths)
            test_x_size = RefDs.PixelSpacing[0]
            test_y_size = RefDs.PixelSpacing[1]
            test_FOV_X = test_cols*test_x_size
            test_FOV_Y = test_rows*test_y_size
            test_FOV_X_min = -test_FOV_X/2.0
            test_FOV_X_max = test_FOV_X/2.0
            test_FOV_Y_min = -test_FOV_Y/2.0
            test_FOV_Y_max = test_FOV_Y/2.0


            test_input = np.zeros((test_cols,test_rows,test_slices), dtype=np.float32)
            test_locations = np.zeros((test_slices))


            for idx, filename in enumerate(list_paths):
                if ".dcm" in filename.lower():  # check whether the file's DICOM
                    RefDs = dicom.read_file(os.path.join(PathDicom,filename))
                    test_input_aux = ((RefDs.pixel_array*RefDs.RescaleSlope)+RefDs.RescaleIntercept) 
                    test_input[:,:,idx] = np.reshape(test_input_aux,(test_cols,test_rows))
                    test_locations[idx] = np.array(RefDs.ImagePositionPatient)[2]

            # re-order
            order_idx = np.argsort(test_locations)
            test_input = test_input[:,:,order_idx]

            test_z_size = np.abs(loc_max-loc_min)/float(len(list_paths))
            test_FOV_Z = test_slices*test_z_size



            Volume_size_INPUT = [test_cols, test_rows, test_slices]
            Voxel_size_INPUT = [test_x_size, test_y_size, test_z_size]

            image_position_patient = np.array(RefDs.ImagePositionPatient)


            if IS_CT:
                # Remove couch
                HU_Air_limit = -1000
                B_n_d = 4
                B_n_e = 2
                B_d_d = 15
                B_d_e = 13
                limits_ct_use = [0,0]
                corr_disk = 6

                couch_mask = DH.CT_couch_removal_mask_multicore(test_input, 
                                                                   samples_use = 20, 
                                                                   n_corr_disk = corr_disk, 
                                                                   limits_CT = limits_ct_use,
                                                                   B_n_dilatation_disk_rad = B_n_d,
                                                                   B_n_erosion_disk_rad = B_n_e,
                                                                   B_d_dilatation_disk_rad = B_d_d,
                                                                   B_d_erosion_disk_rad = B_d_e) 


                for z_idx in range(0,test_input.shape[2]):
                    test_input[:,:,z_idx][couch_mask] = HU_Air_limit               

                # Change units from Housefield to mu
                test_input = ((test_input/1000.0)+1)*DH.mu_agua_120kev/10.0


            # Normalize Size
            test_input_norm, test_limits = DH.normalize_volume_single(test_input,
                                                                         Voxel_size_INPUT, 
                                                                         Objective_size, 
                                                                         Voxel_size_NORM,
                                                                        order_interpol=1)
            post_scaling = 2.0
            test_input_norm_scale = np.zeros((int(test_input_norm.shape[0]/int(post_scaling)),
                                            int(test_input_norm.shape[1]/int(post_scaling)),
                                                int(test_input_norm.shape[2]/int(post_scaling))))

            test_input_norm_scale = ndimage.interpolation.zoom(input=test_input_norm, zoom=[1/post_scaling,1/post_scaling,1/post_scaling], order=2)

            input_net_size = [int(Objective_size[0]/post_scaling),int(Objective_size[1]/post_scaling),int(Objective_size[2]/post_scaling)]

            test_scale_limits = [int(test_limits[0]/2), int(test_limits[1]/2)]

            if IS_CT:
                LIM_SUP_CT = (((HU_Bone_upper_limit/1000.0)+1)*DH.mu_agua_120kev/10.0) 
                LIM_INF_CT = (((HU_fat_lower_limit/1000.0)+1)*DH.mu_agua_120kev/10.0) 

                test_input_norm_scale[test_input_norm_scale<LIM_INF_CT]=LIM_INF_CT
                test_input_norm_scale[test_input_norm_scale>LIM_SUP_CT]=LIM_SUP_CT

            test_input_norm_scale = (test_input_norm_scale - test_input_norm_scale.min()) / (test_input_norm_scale.max() - test_input_norm_scale.min())

            # Save
            if IS_NAC_PT:
                NAC_PET_images[img_count,:,:,:] = np.copy(test_input_norm_scale)
                test_scale_limits_list.append(test_scale_limits)
                COPY_NAC_OK = True

            if IS_CT:

                CT_OBJ_images[img_count,:,:,:] = np.copy(test_input_norm_scale)
                COPY_CT_OK = True

    if not COPY_NAC_OK:
        print('Missing NAC PET scan. Aborting')
    if not COPY_CT_OK:
        print('Missing CT scan. Aborting')

    if COPY_NAC_OK and COPY_CT_OK:

        print('Image limits [ %d ; %d ]'%(test_scale_limits_list[img_count][0],
                                         test_scale_limits_list[img_count][1]))

        slice_X = 64
        slice_Y = 64
        slice_Z = 128

        fig, ax = plt.subplots(nrows=2, ncols=3, dpi=50, sharex=True, sharey=True)
        ax[0][0].imshow(NAC_PET_images[img_count,slice_X,:,:], aspect='equal', vmin=0.0, vmax=1.0)
        ax[0][0].axis('off')
        ax[0][1].imshow(NAC_PET_images[img_count,:,slice_Y,:], aspect='equal', vmin=0.0, vmax=1.0)
        ax[0][1].axis('off')
        ax[0][2].imshow(NAC_PET_images[img_count,:,:,slice_Z], aspect='equal', vmin=0.0, vmax=1.0)
        ax[0][2].axis('off')
        ax[1][0].imshow(CT_OBJ_images[img_count,slice_X,:,:], aspect='equal', vmin=0.0, vmax=1.0)
        ax[1][0].axis('off')
        ax[1][1].imshow(CT_OBJ_images[img_count,:,slice_Y,:], aspect='equal', vmin=0.0, vmax=1.0)
        ax[1][1].axis('off')
        ax[1][2].imshow(CT_OBJ_images[img_count,:,:,slice_Z], aspect='equal', vmin=0.0, vmax=1.0)
        ax[1][2].axis('off')
        plt.tight_layout()
        plt.plot()
        plt.show()

        img_count += 1

# hold
num_test_samples = img_count
NAC_PET_images = NAC_PET_images[:num_test_samples,:,:,:]
CT_OBJ_images = CT_OBJ_images[:num_test_samples,:,:,:]
test_scale_limits_list = test_scale_limits_list[:num_test_samples]



In [ ]:
slice_X = 64
slice_Y = 64
slice_Z = 128
img_count = 0
fig, ax = plt.subplots(nrows=2, ncols=3, dpi=150, sharex=True, sharey=True)
ax[0][0].imshow(NAC_PET_images[img_count,slice_X,:,:], aspect='equal', vmin=0.0, vmax=1.0)
ax[0][0].axis('off')
ax[0][1].imshow(NAC_PET_images[img_count,:,slice_Y,:], aspect='equal', vmin=0.0, vmax=1.0)
ax[0][1].axis('off')
ax[0][2].imshow(NAC_PET_images[img_count,:,:,slice_Z], aspect='equal', vmin=0.0, vmax=1.0)
ax[0][2].axis('off')
ax[1][0].imshow(CT_OBJ_images[img_count,slice_X,:,:], aspect='equal', vmin=0.0, vmax=1.0)
ax[1][0].axis('off')
ax[1][1].imshow(CT_OBJ_images[img_count,:,slice_Y,:], aspect='equal', vmin=0.0, vmax=1.0)
ax[1][1].axis('off')
ax[1][2].imshow(CT_OBJ_images[img_count,:,:,slice_Z], aspect='equal', vmin=0.0, vmax=1.0)
ax[1][2].axis('off')
plt.tight_layout()
plt.plot()
plt.show()

# Compute Test

### Compute whole volumes

In [ ]:
CT_SYNTH_images = np.zeros((num_test_samples,voxels_X_Full,voxels_Y_Full,voxels_Z_Full))

SEGMENTED_images = np.zeros((num_test_samples,voxels_X_Full,voxels_Y_Full,voxels_Z_Full))

MARGIN = 3
data_size = [128,128,256]

for idx_img in range(num_test_samples):


    out_list = GAN.train_support.compute_whole_volume(NAC_PET_images[idx_img,:,:,:].astype(np.float32), 
                                                          input_size, 
                                                          synth_model, 
                                                          np.copy(test_scale_limits_list[idx_img]), 
                                                          MARGIN, 
                                                          sample_num=idx_img, 
                                                          segm_net=SEGM_NET, 
                                                          s_model=synth_model, 
                                                          criticize=False, 
                                                          d_model='', 
                                                          slice_val_cap_use=1.0)

    if SEGM_NET:
        CT_synth_aux = out_list[0]
        segmentation_aux = out_list[1]
    else:
        CT_synth_aux = out_list
    # Save
    CT_SYNTH_images[idx_img,:,:,:] = np.copy(CT_synth_aux)
    if SEGM_NET:
        aux_segm = np.argmax(np.copy(segmentation_aux), axis = -1)
        aux_segm[:,:,:test_scale_limits_list[idx_img][0]]=3
        aux_segm[:,:,test_scale_limits_list[idx_img][1]:]=3
        SEGMENTED_images[idx_img,:,:,:] = aux_segm


# Save synthetic CT samples
np.save(os.path.join(OUTPUT_PATH,'CT_SYNTH_images'), CT_SYNTH_images)
np.save(os.path.join(OUTPUT_PATH,'SEGMENTED_images'), SEGMENTED_images)
    

In [ ]:
X_out = int(data_size[0]/2)
Y_out = int(data_size[1]/2)
Z_out = int(data_size[2]/2)
IMG_SHOW =  np.random.randint(NAC_PET_images.shape[0])
print(IMG_SHOW)

plt.figure(dpi=300)
ax = plt.subplot(1,3,1)
ax.imshow(NAC_PET_images[IMG_SHOW, :,:,Z_out])
plt.axis('off')
ax = plt.subplot(1,3,2)
ax.imshow(NAC_PET_images[IMG_SHOW, :,Y_out,:])
plt.axis('off')
ax = plt.subplot(1,3,3)
ax.imshow(NAC_PET_images[IMG_SHOW, X_out,:,:])
plt.tight_layout()
plt.axis('off')
plt.draw()
plt.savefig(os.path.join(OUTPUT_PATH, 'NAC_PET_input'))
plt.plot()

ct_vmax=CT_OBJ_images.max()
ct_vmin=CT_OBJ_images.min()

plt.figure(dpi=300)
ax = plt.subplot(1,3,1)
ax.imshow(CT_SYNTH_images[IMG_SHOW, :,:,Z_out], vmax=ct_vmax, vmin=ct_vmin)
plt.axis('off')
ax = plt.subplot(1,3,2)
ax.imshow(CT_SYNTH_images[IMG_SHOW,:,Y_out,:], vmax=ct_vmax, vmin=ct_vmin)
plt.axis('off')
ax = plt.subplot(1,3,3)
ax.imshow(CT_SYNTH_images[IMG_SHOW,X_out,:,:], vmax=ct_vmax, vmin=ct_vmin)
plt.tight_layout()
plt.axis('off')
plt.draw()
plt.savefig(os.path.join(OUTPUT_PATH, 'sCT'))
plt.plot()


plt.figure(dpi=300)
ax = plt.subplot(1,3,1)
ax.imshow(CT_OBJ_images[IMG_SHOW,:,:,Z_out], vmax=ct_vmax, vmin=ct_vmin)
plt.axis('off')
ax = plt.subplot(1,3,2)
ax.imshow(CT_OBJ_images[IMG_SHOW,:,Y_out,:], vmax=ct_vmax, vmin=ct_vmin)
plt.axis('off')
ax = plt.subplot(1,3,3)
ax.imshow(CT_OBJ_images[IMG_SHOW,X_out,:,:], vmax=ct_vmax, vmin=ct_vmin)
plt.tight_layout()
plt.axis('off')
plt.draw()
plt.savefig(os.path.join(OUTPUT_PATH, 'objective_CT'))
plt.plot()


if SEGM_NET:
    plt.figure(dpi=300)
    ax = plt.subplot(1,3,1)
    ax.imshow(SEGMENTED_images[IMG_SHOW,:,:,Z_out])
    plt.axis('off')
    ax = plt.subplot(1,3,2)
    ax.imshow(SEGMENTED_images[IMG_SHOW,:,Y_out,:])
    plt.axis('off')
    ax = plt.subplot(1,3,3)
    ax.imshow(SEGMENTED_images[IMG_SHOW,X_out,:,:])
    plt.tight_layout()
    plt.axis('off')
    plt.draw()
    plt.savefig(os.path.join(OUTPUT_PATH, 'Tissue_Clases'))
    plt.plot()

plt.show()


# Compute Metrics

In [ ]:
# Number of slices skipped at the begining and end of the sample
MARGEN_CALC = 16

# Create dataframe
col_names =  ['SSIN', 'MSE_Whole', 'MSE_RoI', 'PSNR_Whole', 'PSNR_RoI', 'MAE', 'MAE_HU', 'NMSE_Whole', 'NMSE_RoI', 'NCC_Whole', 'NCC_RoI']
df_meassurements = pd.DataFrame(columns=col_names) 


# Compute mertic for each sample
for idx_img in range(num_test_samples):

    limits_act = test_scale_limits_list[idx_img]

    
    X_in = np.squeeze(CT_OBJ_images[idx_img,:,:,:])
    X_in = np.expand_dims(X_in, axis=-1)
    X_in = X_in[:,:,limits_act[0]+MARGEN_CALC:limits_act[1]-MARGEN_CALC,:]
    X_in_nonZero = X_in[X_in>0.1]
    
    Y_in = np.squeeze(CT_SYNTH_images[idx_img,:,:,:])
    Y_in = np.expand_dims(Y_in, axis=-1)
    Y_in = Y_in[:,:,limits_act[0]+MARGEN_CALC:limits_act[1]-MARGEN_CALC,:]
    Y_in_nonZero = Y_in[X_in>0.1]

    # SSIM
    ssim_aux = measure.compare_ssim(X_in, Y_in, sigma=5, multichannel=True)
    # MSE
    mse_whole_aux = measure.compare_mse(X_in, Y_in)
    mse_RoI_aux = measure.compare_mse(X_in_nonZero, Y_in_nonZero)
    # PSNR
    psnr_whole_aux = measure.compare_psnr(X_in, Y_in)
    psnr_RoI_aux = measure.compare_psnr(X_in_nonZero, Y_in_nonZero)
    # MAE
    mae_aux = np.sum(np.divide(np.abs(X_in_nonZero-Y_in_nonZero),X_in_nonZero))/(X_in_nonZero.size)
    # NMSE
    nmse_whole_aux = np.sqrt(np.sum((X_in-Y_in)**2))/(np.sqrt(np.sum(X_in**2)))
    nmse_RoI_aux = np.sqrt(np.sum((X_in_nonZero-Y_in_nonZero)**2))/(np.sqrt(np.sum(X_in_nonZero**2)))
    # NCC
    X_in_zm = X_in-np.mean(X_in)
    Y_in_zm = Y_in-np.mean(Y_in)
    ncc_whole_aux = (np.sum(X_in_zm*Y_in_zm)/X_in.size) / (np.std(X_in)*np.std(Y_in))
    
    X_in_zm = X_in_nonZero-np.mean(X_in_nonZero)
    Y_in_zm = Y_in_nonZero-np.mean(Y_in_nonZero)
    ncc_RoI_aux = (np.sum(X_in_zm*Y_in_zm)/X_in_nonZero.size) / (np.std(X_in_nonZero)*np.std(Y_in_nonZero))
    
    
    # Convert to HU
    ct_out_scale = (X_in_nonZero*(LIM_SUP_CT-LIM_INF_CT))+LIM_INF_CT
    X_in_nonZero_HU = (((ct_out_scale*10.0)/DH.mu_agua_120kev)-1)*1000.0

    ct_out_scale = (Y_in_nonZero*(LIM_SUP_CT-LIM_INF_CT))+LIM_INF_CT
    Y_in_nonZero_HU = (((ct_out_scale*10.0)/DH.mu_agua_120kev)-1)*1000.0

    mae_HU_aux = np.sum((np.abs(X_in_nonZero_HU-Y_in_nonZero_HU)))/(X_in_nonZero_HU.size)
    
    
    df_meassurements.loc[len(df_meassurements)] = [ssim_aux, 
                                                   mse_whole_aux, 
                                                   mse_RoI_aux, 
                                                   psnr_whole_aux, 
                                                   psnr_RoI_aux, 
                                                   mae_aux, 
                                                   mae_HU_aux,
                                                   nmse_whole_aux, 
                                                   nmse_RoI_aux, 
                                                   ncc_whole_aux, 
                                                   ncc_RoI_aux]

# Save dataframe
df_meassurements.to_pickle(os.path.join(OUTPUT_PATH,'meassurements_dataframe.pkl'))

In [ ]:
print('Mean metric values:')
print(df_meassurements.mean())

In [ ]:
plt.figure(dpi=200)

ax = plt.subplot(1,3,1)
df_meassurements[['PSNR_RoI']].boxplot()
plt.ylabel('PSNR (db)')
plt.ylim([14.0, 24.0])

ax = plt.subplot(1,3,2)
df_meassurements[['MAE_HU']].boxplot()
plt.ylabel('MAE (HU)')
plt.ylim([60.0, 160.0])


ax = plt.subplot(1,3,3)
df_meassurements[['NCC_Whole']].boxplot()
plt.ylabel('NCC (-)')
plt.ylim([0.45, 1.0])

plt.tight_layout()
plt.show()


# Save as DICOM images

Note that the produced DICOM format is only to facilitate viewing the result with a DICOM reader program such as "Slicer". It should not be regarded as a valid medical image of any nature.

### Convert CT to HU units

In [ ]:
CT_SYNTH_images_HU = np.zeros_like(CT_SYNTH_images)

for idx_img in range(num_test_samples):
    
    # Get current volume
    CT_synth_in = np.squeeze(CT_SYNTH_images[idx_img,:,:,:])
    
    # Voxel dynamic range limits
    LIM_INF_CT = (((HU_fat_lower_limit/1000.0)+1)*DH.mu_agua_120kev/10.0)
    LIM_SUP_CT = (((HU_Bone_upper_limit/1000.0)+1)*DH.mu_agua_120kev/10.0)
    
    # Change voxel scale
    ct_out_scale = (CT_synth_in*(LIM_SUP_CT-LIM_INF_CT))+LIM_INF_CT
    ct_HU = (((ct_out_scale*10.0)/DH.mu_agua_120kev)-1)*1000.0

    CT_SYNTH_images_HU[idx_img,:,:,:] = np.copy(ct_HU)

### Save as DICOM

In [ ]:
for idx_img in range(num_test_samples):
    
    # Get current volume
    CT_synth_in = np.squeeze(CT_SYNTH_images_HU[idx_img,:,:,:])
    # Get current sample name
    SAMPLE_NAME = DicomNameList[idx_img]
    
    # Set image to standard CT size

    # Netwoerk voxel size
    X_VOX_SIZE = Voxel_size_NORM[0]*2
    Y_VOX_SIZE = Voxel_size_NORM[1]*2
    Z_VOX_SIZE = Voxel_size_NORM[2]*2
    # Objective CT voxel size
    X_CT_VOX_SIZE = 1.367
    Y_CT_VOX_SIZE = 1.367
    Z_CT_VOX_SIZE = 3.27

    X_zoom = X_VOX_SIZE/X_CT_VOX_SIZE
    Y_zoom = Y_VOX_SIZE/Y_CT_VOX_SIZE
    Z_zoom = Z_VOX_SIZE/Z_CT_VOX_SIZE

    # Resample
    ct_HU_resampled = np.copy(CT_synth_in)
    ct_HU_resampled = ndimage.interpolation.zoom(input=ct_HU_resampled, zoom=[X_zoom,Y_zoom,Z_zoom], order=1)
    
    # SAve
    DH.saveDICOM_sliced_CT(ct_HU_resampled, 
                           [X_CT_VOX_SIZE, Y_CT_VOX_SIZE, Z_CT_VOX_SIZE], 
                           image_position_patient,
                           OUTPUT_PATH, 
                           SAMPLE_NAME,
                           patient_ID_in = SAMPLE_NAME,
                           StudyInstanceUID = ('1.3.6.1.4.1.14519.5.2.1.3320.3273.2314458152346821195388%d'%np.random.randint(low=63169983, high=93169983)),
                           SeriesInstanceUID = ('1.3.6.1.4.1.14519.5.2.1.3320.3273.330516103388011054891%d'%np.random.randint(low=344582212, high=944582212)),
                           TYPE_INT = np.int16, 
                           vervose = True)